# Fixstars Amplify AEによる数独ソルバー

## イジングマシンの設定

In [1]:
from amplify import Solver
from amplify.client import FixstarsClient
from dotenv import load_dotenv
import os

#.env ファイルをロードして環境変数へ反映
load_dotenv(override=True)

client = FixstarsClient()
client.token = os.getenv('TOKEN')
client.parameters.timeout = 1000 #タイムアウト1秒

solver = Solver(client)

## 関数類の定義

In [2]:
import numpy as np
from amplify import (
    BinaryQuadraticModel,
    BinarySymbolGenerator,
    decode_solution,
    sum_poly
)
from amplify.constraint import one_hot

def print_sudoku(data):
    #数独データの表示関数
    for i in range(len(data)):
        line = ""
        if i == 3 or i == 6:
            print("---------------------")
        for j in range(len(data[i])):
            if j == 3 or j == 6:
                line += "| "
            line += str(data[i][j]) + " "
        print(line)

def solve(sudoku):
    #QUBO模型の作成
    gen = BinarySymbolGenerator()
    q = gen.array(9, 9, 9)
    
    #与えられた数独データの変換(確定している数字を埋め、あり得ない候補を除外)
    for i, j in zip(*np.where(sudoku != 0)):
        k = sudoku[i, j] - 1
        q[i, :, k] = 0
        q[:, j, k] = 0
        q[i, j, :] = 0
        for m in range(9):
            q[(3 * (i // 3) + m // 3), (3 * (j // 3) + m % 3), k] = 0
        q[i, j, k] = 1
    
    #一つのマスには一つの数字しか入らない制約条件
    num_constraints = [one_hot(q[i, j, :]) for i in range(9) for j in range(9)]
    #各行,列には同じ数字が入らない制約条件
    row_constraints = [one_hot(q[i, :, k]) for i in range(9) for k in range(9)]
    col_constraints = [one_hot(q[:, j, k]) for j in range(9) for k in range(9)]
    #3x3ブロック内には同じ数字が入らない制約条件
    block_constraints = [
        one_hot(sum([q[i + m // 3, j + m % 3, k] for m in range(9)]))
        for i in range(0, 9, 3)
        for j in range(0, 9, 3)
        for k in range(9)
    ]
    
    #制約条件の合計
    constraints = (
        sum(num_constraints)
        + sum(row_constraints)
        + sum(col_constraints)
        + sum(block_constraints)
    )
    
    #モデル作成
    model = BinaryQuadraticModel(constraints)
    
    #計算
    result = solver.solve(model)
    if len(result) == 0:
        raise RuntimeError("Some of the constraints are not satisfied.")
    
    #結果の整形
    values = result[0].values
    q_values = q.decode(values)
    answer = np.array([np.where(np.array(q_values[i]) != 0)[1] + 1 for i in range(9)])
    return answer

## 問題の定義及び演算

In [3]:
question = np.array(
    [
        [3,0,0,0,1,0,0,0,6],
        [1,0,5,0,0,0,7,0,2],
        [0,0,0,5,0,8,0,0,0],
        [0,0,3,0,0,0,9,0,0],
        [5,9,0,0,7,0,0,2,4],
        [7,0,1,9,0,4,8,0,5],
        [0,0,0,3,0,2,0,0,0],
        [0,0,4,7,0,1,5,0,0],
        [0,0,0,0,6,0,0,0,0]
    ]
)

print('問題')
print_sudoku(question)
print('解答')
answer = solve(question)
print_sudoku(answer)

問題
3 0 0 | 0 1 0 | 0 0 6 
1 0 5 | 0 0 0 | 7 0 2 
0 0 0 | 5 0 8 | 0 0 0 
---------------------
0 0 3 | 0 0 0 | 9 0 0 
5 9 0 | 0 7 0 | 0 2 4 
7 0 1 | 9 0 4 | 8 0 5 
---------------------
0 0 0 | 3 0 2 | 0 0 0 
0 0 4 | 7 0 1 | 5 0 0 
0 0 0 | 0 6 0 | 0 0 0 
解答
3 8 9 | 2 1 7 | 4 5 6 
1 4 5 | 6 3 9 | 7 8 2 
6 7 2 | 5 4 8 | 3 1 9 
---------------------
4 2 3 | 8 5 6 | 9 7 1 
5 9 8 | 1 7 3 | 6 2 4 
7 6 1 | 9 2 4 | 8 3 5 
---------------------
9 5 6 | 3 8 2 | 1 4 7 
2 3 4 | 7 9 1 | 5 6 8 
8 1 7 | 4 6 5 | 2 9 3 
